<a href="https://colab.research.google.com/github/AndrewZou/GCP-Composer-Airflow-Kubenertes-Environment-Instance/blob/master/github_trend_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GitHub on Hacker News trends analysis

In [5]:
from __future__ import print_function
import pandas as pd

## Input parameters

In [4]:
project_id = "airflow-pipe"
process_date = "2019-12-01"
process_date_nodash = "20191201"

## Exploratory Data Analysis

## Github activity data
- Link: [Data](https://bigquery.cloud.google.com/table/githubarchive:day.20181230) - [More info](https://blog.github.com/2017-01-19-github-data-ready-for-you-to-explore-with-bigquery/)

### Different event type in Gihub activity
- [Event Types & Payloads](https://developer.github.com/v3/activity/events/types/) explaination

In [6]:
query = """
SELECT 
  type,
  COUNT(*) AS cnt
FROM `githubarchive.day.{0}` 
GROUP BY 1
ORDER BY 2 DESC
""".format(process_date_nodash)

print (query)

df = pd.read_gbq(query, project_id=project_id, dialect='standard')
df.head(20)


SELECT 
  type,
  COUNT(*) AS cnt
FROM `githubarchive.day.20191201` 
GROUP BY 1
ORDER BY 2 DESC

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=725825577420-unm2gnkiprugilg743tkbig250f4sfsj.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=PxIz3BwCaEEl1d54kodfGP0eaXLV4j&prompt=consent&access_type=offline
Enter the authorization code: 4/2AFDG69hDjc3Biz2KCooqhqu-B7hqYxhtASSWCUsiosdB4idmkfUucQ


,type,cnt
0,PushEvent,765436
1,CreateEvent,216967
2,PullRequestEvent,95126
3,WatchEvent,86790
4,IssueCommentEvent,60391
5,DeleteEvent,35075
6,IssuesEvent,32850
7,ForkEvent,31175
8,PullRequestReviewCommentEvent,13570
9,GollumEvent,5463


### Top 10 repos with the most comments in their issues
- __IssueCommentEvent__: Triggered when an issue comment is created, edited, or deleted.

In [7]:
query = """
SELECT 
  repo.name,
  COUNT(*) AS cnt
FROM `githubarchive.day.{0}`
WHERE type IN ( 'IssueCommentEvent')
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
""".format(process_date_nodash)

print (query)

df = pd.read_gbq(query, project_id=project_id, dialect='standard')
df.head(20)


SELECT 
  repo.name,
  COUNT(*) AS cnt
FROM `githubarchive.day.20191201`
WHERE type IN ( 'IssueCommentEvent')
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10



,name,cnt
0,openshift/template-service-broker-operator,555
1,google-map-react/google-map-react,434
2,PennyDreadfulMTG/perf-reports,385
3,google-test/signcla-probe-repo,384
4,rust-lang/rust,262
5,openshift/origin,223
6,mottox2/til,212
7,NixOS/nixpkgs,193
8,openshift/oc,172
9,kubernetes/kubernetes,160


### Top 10 repos by stars and fork event

In [8]:
query = """
SELECT 
  repo.name,
  SUM(IF(type='WatchEvent', 1, NULL)) AS stars,
  SUM(IF(type='ForkEvent',  1, NULL)) AS forks,
  COUNT(*) AS cnt
FROM `githubarchive.day.{0}`
WHERE type IN ('WatchEvent','ForkEvent')
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
""".format(process_date_nodash)

print (query)

df = pd.read_gbq(query, project_id=project_id, dialect='standard')
df.head(20)


SELECT 
  repo.name,
  SUM(IF(type='WatchEvent', 1, NULL)) AS stars,
  SUM(IF(type='ForkEvent',  1, NULL)) AS forks,
  COUNT(*) AS cnt
FROM `githubarchive.day.20191201`
WHERE type IN ('WatchEvent','ForkEvent')
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10



,name,stars,forks,cnt
0,evil-huawei/evil-huawei,698,65,763
1,jakubroztocil/httpie,453,8,461
2,sherlock-project/sherlock,294,14,308
3,ix64/unlock-music,233,34,267
4,timqian/chinese-independent-blogs,191,26,217
5,wesbos/beginner-javascript,186,32,218
6,521xueweihan/HelloGitHub,149,20,169
7,alyssaxuu/flowy,124,7,131
8,zhihu-huawei251/zhihu-huawei251,122,8,130
9,fatih/errwrap,120,3,123


## Hacker News data
- Link: [Data](https://bigquery.cloud.google.com/table/bigquery-public-data:hacker_news.full) - [More info](https://medium.com/@hoffa/hacker-news-on-bigquery-now-with-daily-updates-so-what-are-the-top-domains-963d3c68b2e2)

### Top domains shared in Hacker News
- Domain with higher score are more likely to make it to the front page.
- __nytimes__ has the highest average score.

In [9]:
query = """
SELECT 
  REGEXP_EXTRACT(url, '//([^/]*)/?') as domain,
  AVG(score) as avg_score,
  COUNT(*) AS cnt
FROM `bigquery-public-data.hacker_news.full`
WHERE url!='' 
AND EXTRACT(DATE FROM timestamp)="{0}"
GROUP BY 1
ORDER BY 3 DESC 
LIMIT 10
""".format(process_date)

print (query)

df = pd.read_gbq(query, project_id=project_id, dialect='standard')
df.head(20)


SELECT 
  REGEXP_EXTRACT(url, '//([^/]*)/?') as domain,
  AVG(score) as avg_score,
  COUNT(*) AS cnt
FROM `bigquery-public-data.hacker_news.full`
WHERE url!='' 
AND EXTRACT(DATE FROM timestamp)="2019-12-01"
GROUP BY 1
ORDER BY 3 DESC 
LIMIT 10



,domain,avg_score,cnt
0,medium.com,6.588235,34
1,github.com,16.843750,32
2,www.nytimes.com,6.782609,23
3,www.youtube.com,12.166667,18
4,en.wikipedia.org,9.818182,11
5,www.bbc.com,46.909091,11
6,twitter.com,39.444444,9
7,www.theguardian.com,6.142857,7
8,www.washingtonpost.com,28.666667,6
9,www.npr.org,20.500000,6


### What domains have the best chance of getting more than 40 upvotes?
- Certainly Hacker News likes content hosted on sites like github.com and the nytimes.

In [10]:
query = """
SELECT 
  REGEXP_EXTRACT(url, '//([^/]*)/?') as domain,
  COUNTIF(score>40) as score_gt_40,
  COUNT(*) AS cnt
FROM `bigquery-public-data.hacker_news.full`
WHERE url!='' 
AND EXTRACT(DATE FROM timestamp)="{0}"
GROUP BY 1
ORDER BY 2 DESC 
LIMIT 10
""".format(process_date)

print (query)

df = pd.read_gbq(query, project_id=project_id, dialect='standard')
df.head(20)


SELECT 
  REGEXP_EXTRACT(url, '//([^/]*)/?') as domain,
  COUNTIF(score>40) as score_gt_40,
  COUNT(*) AS cnt
FROM `bigquery-public-data.hacker_news.full`
WHERE url!='' 
AND EXTRACT(DATE FROM timestamp)="2019-12-01"
GROUP BY 1
ORDER BY 2 DESC 
LIMIT 10



,domain,score_gt_40,cnt
0,www.bbc.com,3,11
1,www.ams.org,2,2
2,www.wsj.com,2,5
3,github.com,2,32
4,medium.com,1,34
5,blog.plover.com,1,1
6,www.washingtonpost.com,1,6
7,twitter.com,1,9
8,www.reuters.com,1,4
9,www.abc.net.au,1,2


### Top 10 Hacker news stories from Github by highest score

In [11]:
query = """
SELECT     
  `by` AS submitter,
  id as story_id,
  REGEXP_EXTRACT(url, "(https?://github.com/[^/]*/[^/#?]*)") as url,
  SUM(score) as score
FROM
  `bigquery-public-data.hacker_news.full`
WHERE
  type = 'story'
  AND EXTRACT(DATE FROM timestamp)='{0}' 
  AND url LIKE '%https://github.com%'
  AND url NOT LIKE '%github.com/blog/%'
GROUP BY  
  submitter,
  story_id,
  url
ORDER BY score DESC
""".format(process_date)

print (query)

df = pd.read_gbq(query, project_id=project_id, dialect='standard')
df.head(10)


SELECT     
  `by` AS submitter,
  id as story_id,
  REGEXP_EXTRACT(url, "(https?://github.com/[^/]*/[^/#?]*)") as url,
  SUM(score) as score
FROM
  `bigquery-public-data.hacker_news.full`
WHERE
  type = 'story'
  AND EXTRACT(DATE FROM timestamp)='2019-12-01' 
  AND url LIKE '%https://github.com%'
  AND url NOT LIKE '%github.com/blog/%'
GROUP BY  
  submitter,
  story_id,
  url
ORDER BY score DESC



,submitter,story_id,url,score
0,rsapkf,21674729,https://github.com/jakubroztocil/httpie,367
1,ingve,21675146,https://github.com/oreillymedia/etudes-for-erlang,63
2,osprojects,21676256,https://github.com/adnanh/webhook,32
3,santiagobasulto,21677123,https://github.com/santiagobasulto/python-hack...,8
4,vinnyglennon,21678078,https://github.com/trimstray/the-book-of-secre...,7
5,jidiculous,21673453,https://github.com/jidicula/pretinder,7
6,based2,21675141,https://github.com/Eerovil/TrackLater,5
7,orn0t,21673050,https://github.com/orn0t/teamcity-locustio,4
8,hongzi,21675284,https://github.com/blaisewang/img2latex-mathpix,3
9,bane,21676267,https://github.com/serejandmyself/cyber,3


## Example Final table: GitHub on Hacker News Trends of 2019-12-01

In [12]:
query = """
WITH github_activity AS (
SELECT   
  repo.name as repo,
  CONCAT('https://github.com/', repo.name) as url,
  SUM(IF(type='WatchEvent', 1, NULL)) AS stars,
  SUM(IF(type='ForkEvent',  1, NULL)) AS forks,
  COUNT(*) AS cnt
FROM `githubarchive.day.{0}`
WHERE type IN ('WatchEvent','ForkEvent')
GROUP BY 1,2
),
hacker_news AS (
SELECT
  EXTRACT(DATE FROM timestamp) as date,
  `by` AS submitter,
  id as story_id,
  REGEXP_EXTRACT(url, "(https?://github.com/[^/]*/[^/#?]*)") as url,
  SUM(score) as score
FROM
  `bigquery-public-data.hacker_news.full`
WHERE
  type = 'story'
  AND EXTRACT(DATE FROM timestamp)='{1}' 
  AND url LIKE '%https://github.com%'
  AND url NOT LIKE '%github.com/blog/%'
GROUP BY 1,2,3,4
)

SELECT
  a.date as date,
  a.url as github_url,
  b.repo as github_repo,
  a.score as hn_score,
  a.story_id as hn_story_id,
  b.stars as stars,
  b.forks as forks
FROM hacker_news as a
LEFT JOIN github_activity as b
ON a.url=b.url
ORDER BY hn_score DESC
LIMIT 10
""".format(process_date_nodash, process_date)

print (query)

df = pd.read_gbq(query, project_id=project_id, dialect='standard')
df.head(10)


WITH github_activity AS (
SELECT   
  repo.name as repo,
  CONCAT('https://github.com/', repo.name) as url,
  SUM(IF(type='WatchEvent', 1, NULL)) AS stars,
  SUM(IF(type='ForkEvent',  1, NULL)) AS forks,
  COUNT(*) AS cnt
FROM `githubarchive.day.20191201`
WHERE type IN ('WatchEvent','ForkEvent')
GROUP BY 1,2
),
hacker_news AS (
SELECT
  EXTRACT(DATE FROM timestamp) as date,
  `by` AS submitter,
  id as story_id,
  REGEXP_EXTRACT(url, "(https?://github.com/[^/]*/[^/#?]*)") as url,
  SUM(score) as score
FROM
  `bigquery-public-data.hacker_news.full`
WHERE
  type = 'story'
  AND EXTRACT(DATE FROM timestamp)='2019-12-01' 
  AND url LIKE '%https://github.com%'
  AND url NOT LIKE '%github.com/blog/%'
GROUP BY 1,2,3,4
)

SELECT
  a.date as date,
  a.url as github_url,
  b.repo as github_repo,
  a.score as hn_score,
  a.story_id as hn_story_id,
  b.stars as stars,
  b.forks as forks
FROM hacker_news as a
LEFT JOIN github_activity as b
ON a.url=b.url
ORDER BY hn_score DESC
LIMIT 10



,date,github_url,github_repo,hn_score,hn_story_id,stars,forks
0,2019-12-01,https://github.com/jakubroztocil/httpie,jakubroztocil/httpie,367,21674729,453.0,8.0
1,2019-12-01,https://github.com/oreillymedia/etudes-for-erlang,oreillymedia/etudes-for-erlang,63,21675146,49.0,2.0
2,2019-12-01,https://github.com/adnanh/webhook,adnanh/webhook,32,21676256,24.0,NaN
3,2019-12-01,https://github.com/santiagobasulto/python-hack...,None,8,21677123,NaN,NaN
4,2019-12-01,https://github.com/jidicula/pretinder,jidicula/pretinder,7,21673453,2.0,NaN
5,2019-12-01,https://github.com/trimstray/the-book-of-secre...,trimstray/the-book-of-secret-knowledge,7,21678078,46.0,10.0
6,2019-12-01,https://github.com/Eerovil/TrackLater,Eerovil/TrackLater,5,21675141,9.0,NaN
7,2019-12-01,https://github.com/orn0t/teamcity-locustio,orn0t/teamcity-locustio,4,21673050,2.0,NaN
8,2019-12-01,https://github.com/blaisewang/img2latex-mathpix,blaisewang/img2latex-mathpix,3,21675284,89.0,13.0
9,2019-12-01,https://github.com/serejandmyself/cyber,None,3,21676267,NaN,NaN


## Python PyPI stats
- The Python Software Foundation provides the raw logs of Python installation activitie
- Link: [Data](https://bigquery.cloud.google.com/table/the-psf:pypi.downloads20181230) - [More info](https://packaging.python.org/guides/analyzing-pypi-package-downloads/)

__Challenge__: 
- Find associated Github stars, fork event, and Hacker News story for top downloads Python packages from pip

### Top 10 downloads packages from pip

In [13]:
query = """
SELECT 
  TIMESTAMP_TRUNC(timestamp, DAY) as day,
  file.project as project,
  COUNT(*) as downloads
FROM `the-psf.pypi.downloads{0}`
WHERE details.installer.name = 'pip'
GROUP BY 1,2
ORDER BY 3 desc
""".format(process_date_nodash)

print (query)

df = pd.read_gbq(query, project_id=project_id, dialect='standard')
df.head(10)


SELECT 
  TIMESTAMP_TRUNC(timestamp, DAY) as day,
  file.project as project,
  COUNT(*) as downloads
FROM `the-psf.pypi.downloads20191201`
WHERE details.installer.name = 'pip'
GROUP BY 1,2
ORDER BY 3 desc



,day,project,downloads
0,2019-12-01 00:00:00+00:00,urllib3,2239374
1,2019-12-01 00:00:00+00:00,six,1955634
2,2019-12-01 00:00:00+00:00,certifi,1659564
3,2019-12-01 00:00:00+00:00,requests,1636200
4,2019-12-01 00:00:00+00:00,python-dateutil,1590385
5,2019-12-01 00:00:00+00:00,botocore,1559959
6,2019-12-01 00:00:00+00:00,idna,1545890
7,2019-12-01 00:00:00+00:00,chardet,1485882
8,2019-12-01 00:00:00+00:00,pip,1417038
9,2019-12-01 00:00:00+00:00,s3transfer,1413075


### What is the number stars and fork event for botocore?

In [14]:
query = """
SELECT 
  repo.name,
  SUM(IF(type='WatchEvent', 1, NULL)) AS stars,
  SUM(IF(type='ForkEvent',  1, NULL)) AS forks,
  COUNT(*) AS cnt
FROM `githubarchive.day.{0}`
WHERE type IN ('WatchEvent','ForkEvent')
AND repo.name LIKE "%botocore%"
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
""".format(process_date_nodash)

print (query)

df = pd.read_gbq(query, project_id=project_id, dialect='standard')
df.head(20)


SELECT 
  repo.name,
  SUM(IF(type='WatchEvent', 1, NULL)) AS stars,
  SUM(IF(type='ForkEvent',  1, NULL)) AS forks,
  COUNT(*) AS cnt
FROM `githubarchive.day.20191201`
WHERE type IN ('WatchEvent','ForkEvent')
AND repo.name LIKE "%botocore%"
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10



,name,stars,forks,cnt
0,boto/botocore,None,1,1


## Resources
- [GitHub data, ready for you to explore with BigQuery](https://blog.github.com/2017-01-19-github-data-ready-for-you-to-explore-with-bigquery/)
- [Hacker News on BigQuery](https://medium.com/@hoffa/hacker-news-on-bigquery-now-with-daily-updates-so-what-are-the-top-domains-963d3c68b2e2)
- [Analyzing PyPI package downloads](https://packaging.python.org/guides/analyzing-pypi-package-downloads/)